In [1]:
!pip install pymongo
from pymongo import MongoClient

In [2]:
host="rhea.isegi.unl.pt"
port="28011"
user="GROUP_11"
password="MTM0MzQ5MzI3NjY4ODM1MzQ2ODc2OTMzMjgyNjQ3NjYyNDcyNjUw"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [3]:
db = client.contracts
eu = db.eu

In [4]:
db.list_collection_names()

['eu', 'cpv', 'iso_codes', 'cpv_codes']

In [5]:
eu.find_one()

{'_id': ObjectId('5e58fe88e3912091230e07a2'),
 'ID_NOTICE_CAN': 20157,
 'TED_NOTICE_URL': 'ted.europa.eu/udl?uri=TED:NOTICE:7-2015:TEXT:EN:HTML',
 'YEAR': 2015,
 'ID_TYPE': 3,
 'DT_DISPATCH': '23-DEC-14',
 'XSD_VERSION': 'R208.S2',
 'CANCELLED': 0,
 'CORRECTIONS': 0,
 'CAE_NAME': 'European Union Agency for Law Enforcement Training (CEPOL)',
 'CAE_ADDRESS': 'Ó utca 27.',
 'CAE_TOWN': 'Budapest',
 'CAE_POSTAL_CODE': 1066,
 'CAE_GPA_ANNEX': 'A1EUN',
 'ISO_COUNTRY_CODE': 'HU',
 'ISO_COUNTRY_CODE_GPA': 'EU',
 'CAE_TYPE': 5,
 'EU_INST_CODE': 'AG',
 'MAIN_ACTIVITY': 'Public Order and Safety',
 'B_ON_BEHALF': 'N',
 'TYPE_OF_CONTRACT': 'S',
 'TAL_LOCATION_NUTS': 'HU101;HU102',
 'B_FRA_AGREEMENT': 'Y',
 'B_FRA_CONTRACT': 'Y',
 'CPV': 79620000,
 'MAIN_CPV_CODE_GPA': 796,
 'ADDITIONAL_CPVS': '79621000---79612000---79610000---79600000---79611000',
 'B_GPA': 'N',
 'GPA_COVERAGE': 2,
 'LOTS_NUMBER': 0,
 'VALUE_EURO': 400000,
 'VALUE_EURO_FIN_1': 400000,
 'VALUE_EURO_FIN_2': 400000,
 'B_EU_FUNDS': 'N'

## Precomputed collection

Due to the fact that, for the dashboard, only a limited set of countries was considered, as well as a limited interval of years and values, the first thing to be done was to filter the "eu" collection in order to only get the documents of interest. Also, since not all fields were used in the queries, it was decied to only project the fields that would be relevant to the problems at hand.

In [6]:
countries = ['NO', 'HR', 'HU', 'CH', 'CZ', 'RO', 'LV', 'GR', 'UK', 'SI', 'LT',
             'ES', 'FR', 'IE', 'SE', 'NL', 'PT', 'PL', 'DK', 'MK', 'DE', 'IT',
             'BG', 'CY', 'AT', 'LU', 'BE', 'FI', 'EE', 'SK', 'MT', 'LI', 'IS']

In [7]:
match = {
    '$match':{
        '$and':[{'YEAR':{'$gte':2008}},{'YEAR':{'$lte':2020}}],
        'VALUE_EURO':{'$lte':100000000},
        'ISO_COUNTRY_CODE':{'$in':countries}
    }
}

project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'NUMBER_OFFERS':True,
        'nCPV':'$CPV',
        'CPV':{'$toString':'$CPV'},
        'ISO_COUNTRY_CODE':True,
        'YEAR':True,
        'AWARD_VALUE_EURO':True,
        'DT_DISPATCH':{'$dateFromString':{'dateString':'$DT_DISPATCH'}},
        'DT_AWARD':{'$dateFromString':{'dateString':'$DT_AWARD'}},
        'B_EU_FUNDS':True,
        'CAE_NAME':True,
        'CAE_ADDRESS':True,
        'CAE_TOWN':True,
        'WIN_NAME':True,
        'ADDRESS':{'$concat':[{'$toString':'$CAE_ADDRESS'},' ',{'$toString':'$CAE_TOWN'}]},
        'PAIR':{'$concat':[{'$toString':'$CAE_NAME'},' with ',{'$toString':'$WIN_NAME'}]}
    }
}

pipeline = [match,project]

list_documents = list(eu.aggregate(pipeline))

In [8]:
db.eu_2.insert_many(list_documents)

In [9]:
db.eu_2.find_one()

{'_id': ObjectId('5ef0a9a735a8d6e890247994'),
 'YEAR': 2015,
 'CAE_NAME': 'European Union Agency for Law Enforcement Training (CEPOL)',
 'CAE_ADDRESS': 'Ó utca 27.',
 'CAE_TOWN': 'Budapest',
 'ISO_COUNTRY_CODE': 'HU',
 'VALUE_EURO': 400000,
 'B_EU_FUNDS': 'N',
 'WIN_NAME': 'Manpower Munkaerő-Szervezési Kft',
 'NUMBER_OFFERS': 1,
 'AWARD_VALUE_EURO': 400000,
 'nCPV': 79620000,
 'CPV': '79620000',
 'DT_DISPATCH': datetime.datetime(2014, 12, 23, 0, 0),
 'DT_AWARD': datetime.datetime(2014, 11, 20, 0, 0),
 'ADDRESS': 'Ó utca 27. Budapest',
 'PAIR': 'European Union Agency for Law Enforcement Training (CEPOL) with Manpower Munkaerő-Szervezési Kft'}

In [22]:
db.eu_2.count_documents({})

3041090

After creating a smaller collection, it was necessary to deal with the fact that there were CPV codes where the fisrt digit was 0 and therefore missing, leading to a miscalculation of the CPV division for some documents in the collection. In order to prevent that fact, all the problematic documents were identified and the CPV codes corrected. Then, the CPV division code was generated, in order to save time on future computations.

In [11]:
db.eu_2.update_many({'$expr':{'$lt':['$nCPV',10000000]}},[{'$set':{'CPV':{'$concat':['0','$CPV']}}}])

In [12]:
db.eu_2.update_many({},[{'$set':{'CPV_DIVISION':{'$substr':['$CPV',0,2]}}}])

In [13]:
db.eu_2.find_one()

{'_id': ObjectId('5ef0a9a735a8d6e890247994'),
 'YEAR': 2015,
 'CAE_NAME': 'European Union Agency for Law Enforcement Training (CEPOL)',
 'CAE_ADDRESS': 'Ó utca 27.',
 'CAE_TOWN': 'Budapest',
 'ISO_COUNTRY_CODE': 'HU',
 'VALUE_EURO': 400000,
 'B_EU_FUNDS': 'N',
 'WIN_NAME': 'Manpower Munkaerő-Szervezési Kft',
 'NUMBER_OFFERS': 1,
 'AWARD_VALUE_EURO': 400000,
 'nCPV': 79620000,
 'CPV': '79620000',
 'DT_DISPATCH': datetime.datetime(2014, 12, 23, 0, 0),
 'DT_AWARD': datetime.datetime(2014, 11, 20, 0, 0),
 'ADDRESS': 'Ó utca 27. Budapest',
 'PAIR': 'European Union Agency for Law Enforcement Training (CEPOL) with Manpower Munkaerő-Szervezési Kft',
 'CPV_DIVISION': '79'}

## ISO Codes: UK problem

Since there was a inconsistency in the collections and the UK was identified differently in the "eu" and "iso_codes" collections, it was necessary to change one of them in order to guarantee that the queries would work correctly.

In order to solve this problem there were two alternatives:
1. changing the ISO_COUNTRY_CODE field from UK to GB in the "eu" collection, and so making the documents in the "eu" collection consistent with, not only the "iso_codes" collection, but also with the proper alpha-2 codes;
2. changing the alpha-2 field from GB to UK in the "iso_codes" collection.

The first oprion is more correct since the the actual alpha-2 code for the United Kingdom is indeed GB, but this means further changes to the code, that was prepared to recieve UK in the ISO_COUNTRY_CODE field. It also means that for any future document inserted, there needs to be a guarantee that where the ISO_COUNTRY_CODE is UK, it is changed to GB.

In [19]:
list(db.eu_2.find({'ISO_COUNTRY_CODE':'UK'}))

[{'_id': ObjectId('5ef0a9a735a8d6e8902479d1'),
  'YEAR': 2009,
  'CAE_NAME': 'RSS',
  'CAE_ADDRESS': 'Bretten Hall, 45 Bush Road',
  'CAE_TOWN': 'Antrim',
  'ISO_COUNTRY_CODE': 'UK',
  'VALUE_EURO': 380547.51,
  'B_EU_FUNDS': 'N',
  'WIN_NAME': 'Northern Microsystems',
  'NUMBER_OFFERS': 6,
  'nCPV': 42931000,
  'CPV': '42931000',
  'DT_DISPATCH': datetime.datetime(2008, 12, 29, 0, 0),
  'DT_AWARD': datetime.datetime(2008, 11, 13, 0, 0),
  'ADDRESS': 'Bretten Hall, 45 Bush Road Antrim',
  'PAIR': 'RSS with Northern Microsystems',
  'CPV_DIVISION': '42'},
 {'_id': ObjectId('5ef0a9a735a8d6e8902479d4'),
  'YEAR': 2009,
  'CAE_NAME': 'Wakefield Metropolitan District Council',
  'CAE_ADDRESS': 'Sport and Active Lifestyles, Town Hall',
  'CAE_TOWN': 'Normanton',
  'ISO_COUNTRY_CODE': 'UK',
  'VALUE_EURO': 130118.75,
  'B_EU_FUNDS': 'N',
  'WIN_NAME': 'XN Leisure Systems Ltd',
  'nCPV': 72000000,
  'CPV': '72000000',
  'DT_DISPATCH': datetime.datetime(2008, 12, 30, 0, 0),
  'DT_AWARD': dateti

In [20]:
db.eu_2.update_many({'ISO_COUNTRY_CODE':'UK'},[{'$set':{'ISO_COUNTRY_CODE':'GB'}}])

In [21]:
list(db.eu_2.find({'ISO_COUNTRY_CODE':'GB'}))

[{'_id': ObjectId('5ef0a9a735a8d6e8902479d1'),
  'YEAR': 2009,
  'CAE_NAME': 'RSS',
  'CAE_ADDRESS': 'Bretten Hall, 45 Bush Road',
  'CAE_TOWN': 'Antrim',
  'ISO_COUNTRY_CODE': 'GB',
  'VALUE_EURO': 380547.51,
  'B_EU_FUNDS': 'N',
  'WIN_NAME': 'Northern Microsystems',
  'NUMBER_OFFERS': 6,
  'nCPV': 42931000,
  'CPV': '42931000',
  'DT_DISPATCH': datetime.datetime(2008, 12, 29, 0, 0),
  'DT_AWARD': datetime.datetime(2008, 11, 13, 0, 0),
  'ADDRESS': 'Bretten Hall, 45 Bush Road Antrim',
  'PAIR': 'RSS with Northern Microsystems',
  'CPV_DIVISION': '42'},
 {'_id': ObjectId('5ef0a9a735a8d6e8902479d4'),
  'YEAR': 2009,
  'CAE_NAME': 'Wakefield Metropolitan District Council',
  'CAE_ADDRESS': 'Sport and Active Lifestyles, Town Hall',
  'CAE_TOWN': 'Normanton',
  'ISO_COUNTRY_CODE': 'GB',
  'VALUE_EURO': 130118.75,
  'B_EU_FUNDS': 'N',
  'WIN_NAME': 'XN Leisure Systems Ltd',
  'nCPV': 72000000,
  'CPV': '72000000',
  'DT_DISPATCH': datetime.datetime(2008, 12, 30, 0, 0),
  'DT_AWARD': dateti